<a href="https://colab.research.google.com/github/TeradaZenichi/dnotools/blob/master/Fluxo_de_carga_radial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fluxo de Carga Radial**

**Nome:** Lucas Zenichi Terada

**Institution:** University of Campinas

In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

## Análise Topológica

In [33]:
def verify(df, bar, aux_bus, aux_net, aux_swt, level, radial, line_type):
  current_bar = 0
  if df.start == bar and df.level == -1:
    current_bar = df.end
  if df.end == bar and df.level == -1:
    current_bar = df.start 
  if current_bar > 0:
    if line_type == 'line':
      net.loc[df.num-1, 'level'] = level
      aux_net.append(int(df.num))
    if line_type == 'switch':
      swt.loc[df.num-1, 'level'] = level
      aux_swt.append(int(df.num))
    if bus.loc[current_bar-1, 'level'] == -1:
      bus.loc[current_bar-1, 'level'] = level
      aux_bus.append(int(current_bar))
    else:
      radial = False
  return aux_bus, aux_net, aux_swt, radial

In [34]:
def radiality():
  (level, buslv, netlv, swtlv) = (0, [], [], [])
  aux = []
  (bus['level'], net['level'], swt['level']) = (np.ones(len(bus), dtype=int)*(-1),
  np.ones(len(net), dtype=int)*(-1), np.ones(len(swt), dtype=int)*(-1))
  radial = True
  for ob in bus.itertuples():
    if ob.bustype == 'feeder':
      bus.loc[ob.Index, 'level'] = 0
      aux.append(int(ob.num)) 
  buslv.append(aux)
  netlv.append([])
  swtlv.append([])
  level = 1
  while radial is True:
    aux_bus = []
    aux_net = []
    aux_swt = []
    for bar in buslv[level-1]:
      for ol in net.itertuples():
        (aux_bus, aux_net, aux_swt, radial) =  verify(net.loc[ol.num-1], bar, aux_bus, aux_net, aux_swt, level, radial, 'line')
      for sw in swt.itertuples():
        if sw.pos == 1:
          (aux_bus, aux_net, aux_swt, radial) = verify(swt.loc[sw.Index], bar, aux_bus, aux_net, aux_swt, level, radial, 'switch')
    if len(aux_bus) > 0:
      buslv.append(aux_bus)
      netlv.append(aux_net)
      swtlv.append(aux_swt)
      level +=1
    else:
      break
  return radial, level, buslv, netlv, swtlv

In [35]:
# Carregar os arquivos dataframe para análise topológica
global bus, net, swt, par
bus = pd.read_csv('bus.csv')
net = pd.read_csv('networks.csv')
swt = pd.read_csv('switches.csv')
par = json.load(open('param.json','r'))

In [36]:
net.r = net.r/1000
net.x = net.x/1000
swt.r = swt.r/1000
swt.x = swt.x/1000

## Fluxo de carga - backward forward sweep

In [37]:
def error_calc(v, Ibus):
  error = 0
  for bar in bus.itertuples():
    if bar.bustype == 'load' and bar.level > 0:
      aux = np.abs(bar.pd+1j*bar.qd - v[bar.num-1]*np.conj(Ibus[bar.num-1]))
      error = np.maximum(aux, error)
  return error

In [38]:
def identify(df, lv):
  if bus.level[int(df.start)-1] == int(lv):
    mul = -1
    return int(df.start), mul
  elif bus.level[int(df.end)-1] == int(lv):
    mul = 1
    return int(df.end), mul
  else:
    return 0

In [39]:
def nodalcurrent(Ibus, df, lv):
  if bus.level[int(df.start)-1] == lv:
    return -Ibus[int(df.start)-1]
  elif bus.level[int(df.end)-1] == lv:
    return Ibus[int(df.end)-1]

In [40]:
def current(bar, Ikm, df, nplus):
  Inear = 0
  if bar == df.start:
    Inear = Ikm[nplus-1]
  elif bar == df.end:
    Inear = -Ikm[nplus-1]
  return Inear

In [41]:
def currente_calc(lv, v, Ibus, Inet, Iswt, buslv, netlv, swtlv, level):
  for n in netlv[lv]:
    Inet[n-1] = nodalcurrent(Ibus, net.loc[n-1], lv)
  for s in swtlv[lv]:
    Iswt[s-1] = nodalcurrent(Ibus, swt.loc[s-1], lv)
  if lv+1 < level:
    for n in netlv[lv]:
      (bar, mul) = identify(net.loc[n-1], lv)
      for nplus in netlv[lv+1]:
          Inet[n-1] = Inet[n-1] + mul*current(bar, Inet, net.loc[nplus-1], nplus)
      for splus in swtlv[lv+1]:
          Inet[n-1] = Inet[n-1] + mul*current(bar, Iswt, swt.loc[splus-1], splus)
    for s in swtlv[lv]:
      (bar, mul) = identify(swt.loc[s-1], lv)
      for splus in swtlv[lv+1]:
        Iswt[s-1] = Iswt[s-1] + mul*current(bar, Iswt, swt.loc[splus-1], splus)
      for nplus in netlv[lv+1]:
        Iswt[s-1] = Iswt[s-1] + mul*current(bar, Inet, net.loc[nplus-1], nplus)
  return Inet, Iswt

In [72]:
def voltage_calc(lv, v, Ibus, Inet, Iswt, buslv, netlv, swtlv, level):
  for n in netlv[lv]:
    if bus.level[net.start[n-1]-1] > bus.level[net.end[n-1]-1]:
      v[net.start[n-1]-1] = v[net.end[n-1]-1] - Inet[n-1]*(net.r[n-1]+1j*net.x[n-1])
    else:
      v[net.end[n-1]-1] = v[net.start[n-1]-1] - Inet[n-1]*(net.r[n-1]+1j*net.x[n-1])
  for s in swtlv[lv]:
      if bus.level[swt.start[s-1]-1] > bus.level[swt.end[s-1]-1]:
        v[swt.start[s-1]-1] = v[swt.end[s-1]-1] - Iswt[s-1]*(swt.r[s-1]+1j*swt.x[s-1])
      else:
        v[swt.end[s-1]-1] = v[swt.start[s-1]-1] - Iswt[s-1]*(swt.r[s-1]+1j*swt.x[s-1])
  return v

In [43]:
def initial_voltage(buslv, level):
  v = np.zeros(len(bus), dtype=complex)
  for bar in buslv[0]:
    v[bar-1] = bus.baseKV[bar-1]/np.sqrt(3)
  for lv in range(1,level):
    for line in netlv[lv]:
      if bus.level[net.start[line-1]-1] < bus.level[net.end[line-1]-1]:
        v[net.end[line-1]-1] = v[net.start[line-1]-1]
      elif bus.level[net.end[line-1]-1] < bus.level[net.start[line-1]-1]:
        v[net.start[line-1]-1] = v[net.end[line-1]-1]
    for switch in swtlv[lv]:
      if bus.level[swt.start[switch-1]-1] < bus.level[swt.end[switch-1]-1]:
        v[swt.end[switch-1]-1] = v[swt.start[switch-1]-1]
      elif bus.level[swt.end[switch-1]-1] < bus.level[swt.start[switch-1]-1]:
        v[swt.start[switch-1]-1] = v[swt.end[switch-1]-1]
  return v

In [44]:
def sweeppowerflow():
  #estados a serem calculados
  Ibus = np.zeros(len(bus), dtype=complex)
  Inet = np.zeros(len(net), dtype=complex)
  Iswt = np.zeros(len(swt), dtype=complex)
  v = initial_voltage(buslv, level)
  error = np.infty
  iteration = 0
  while error > 1e-5 and iteration < 15:
  # Passo 2: Injeção de corrente
    for bar in bus.itertuples():
      if v[bar.num-1] > 0:
        Ibus[bar.num-1] = np.conj((bar.pd+1j*bar.qd)/(par['snom']*v[bar.num-1]))
    # Passo 3: (Backward sweep): Fluxo de corrente para cada level
    for lv in range(level-1,0,-1):
      (Inet, Iswt) = currente_calc(lv, v, Ibus, Inet, Iswt, buslv, netlv, swtlv, level)
    # Passo 4: (Forward sweep:) Tensões nas barras
    for lv in range(1,level):
      v = voltage_calc(lv, v, Ibus, Inet, Iswt, buslv, netlv, swtlv, level)
    error = error_calc(v, Ibus)
    iteration = iteration + 1
  return v, Ibus, Inet, Iswt, error, iteration

## Funções adicionais para fluxo de carga

In [45]:
def loss():
  Ploss = 0.0
  for line in net.itertuples():
    Ploss = Ploss + line.r*(line.Ikm**2)
  for switch in swt.itertuples():
    Ploss = Ploss + switch.r*(switch.Ikm**2)
  return Ploss

In [46]:
def branch_current(Inet, Iswt):
  net['Ikm'] = np.round(np.abs(Inet), decimals=4)
  swt['Ikm'] = np.round(np.abs(Iswt), decimals=4)

In [47]:
def refresh_bus(v):
  v = np.round(np.abs(v), decimals=4)
  bus['v'] = np.abs(v)

In [48]:
def line_power(v, Inet, Iswt):
  S = np.zeros(len(net), dtype=complex)
  for line in net.itertuples():
    S[line.Index] = v[line.start-1]*np.conj(Inet[line.Index])
  net['Pkm'] = np.round(np.real(S), decimals=4)
  net['Qkm'] = np.round(np.imag(S), decimals=4)
  S = np.zeros(len(swt), dtype=complex)
  for line in swt.itertuples():
    S[line.Index] = v[line.start-1]*np.conj(Inet[line.Index])
  swt['Pkm'] = np.round(np.real(S), decimals=4)
  swt['Qkm'] = np.round(np.imag(S), decimals=4)

In [49]:
def states():
  (v, Ibus, Inet, Iswt, error, iteration) = sweeppowerflow()
  line_power(v, Inet, Iswt)
  branch_current(Inet, Iswt)
  refresh_bus(v)

## Funções restrições

In [50]:
def radiality_restriction():
  mod_bus = len(bus)
  mod_net = len(net)
  mod_sub = Counter(bus.bustype)['feeder']
  return 

In [51]:
def voltage_constraint(upper, lower):
  respect = True
  for bar in bus.itertuples():
    if bar.v*np.sqrt(3)/bar.baseKV > upper and bar.level > -1:
      respect = False
      break
    if bar.v*np.sqrt(3)/bar.baseKV < lower and bar.level > -1:
      respect = False
      break
  return respect

In [52]:
def custo(cr, cch, cls):
  custo = 0.0
  for bar in bus.itertuples():
    if bar.level == -1:
      custo = custo + cr*bar.pd
  nchanges = np.sum(np.abs(swt.ini - swt.pos))
  custo = custo + cch*nchanges
  custo = custo + cls*loss()
  return custo

In [123]:
def isolation():
  respect = True
  for bar in bus.itertuples():
    if par['zone_status'][bar.zb] == 0 and bar.v > 0:
      respect = False
  return respect

In [54]:
def bin2int():
  size = len(swt)
  value = 0
  for i in range(size-1,-1,-1):
    value = value + swt.pos[size-i-1]*2**i
  return value

## Restauração

Radial



### Documentação de funções usadas na busca Tabu


radiality()

radiality_restriction()  -- **Não implementado**

voltage_constraint(upper, lower)

custo(cr, cch, cls)

isolation()

**Criar função para potência nos alimentadores**



In [149]:
#Definição de custos:
(cr, cch, cls) = (5.0, 0.2, 0.01)
par['zone_status'] = [1,0,1,1,1,1,1,1,1,1]
BTmax = 3        #argumento da função
sol = np.zeros(len(swt), dtype=int)
swt['pos'] = sol
(radial, level, buslv, netlv, swtlv) = radiality()
states()
best_cost = custo(cr, cch, cls)
iter = 0
bestiter = 0
tabulist = list()

In [151]:
# Busca de soluções vizinhas
while iter-bestiter < BTmax:
  total_solutions = []
  for i in range(0,len(swt)):
    neighbor = sol.copy()
    neighbor[i] = neighbor[i] ^ 1
    int_value = bin2int()
    swt['pos'] = neighbor
    if int_value not in tabulist:
      (radial, level, buslv, netlv, swtlv) = radiality()
      if radial is True:
        states()
        if isolation() is False:
          tabulist.append(int_value)
          break
        elif voltage_constraint(1.05, 0.80) is False:
          tabulist.append(int_value)
          break
        else:
          current_cost = custo(cr, cch, cls)
          total_solutions.append(current_cost)
          if current_cost < best_cost:
            best_sol = neighbor
            best_cost = current_cost
            bestiter = iter
      else:
        tabulist.append(int_value)

  iter += 1
  sol = best_sol
  print(sol,'\n')
  print("iterations: ",iter,'\n')
  print('melhor iteracao: ', bestiter,'\n')
  print('Soluções encontradas: ', total_solutions,'\n')
  print('Melhor solução (valor): ', best_cost)
  print('Lista Tabu: ', tabulist,'\n\n\n')

[0 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1] 

iterations:  1 

melhor iteracao:  0 

Soluções encontradas:  [] 

Melhor solução (valor):  25501.8
Lista Tabu:  [0] 



[0 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1] 

iterations:  2 

melhor iteracao:  0 

Soluções encontradas:  [] 

Melhor solução (valor):  25501.8
Lista Tabu:  [0, 131072] 



[0 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1] 

iterations:  3 

melhor iteracao:  0 

Soluções encontradas:  [] 

Melhor solução (valor):  25501.8
Lista Tabu:  [0, 131072, 181527] 





In [150]:
sol

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])